In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [2]:
from config import key_dict

if 'maps' in key_dict:
    maps_key = key_dict['maps']
else:
    maps_key = api_key

In [3]:
# load csv file

file = "cities.csv"

weather_df = pd.read_csv(file)

weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,isabela,90,PH,1616295319,79,6.7085,121.9711,84.20,4.61
1,1,isangel,98,VU,1616295321,73,-19.5500,169.2667,81.14,12.41
2,2,narsaq,75,GL,1616295321,93,60.9167,-46.0500,32.00,11.50
3,3,shimoda,75,JP,1616295321,100,34.6667,138.9500,66.99,28.77
4,4,lavrentiya,100,RU,1616295322,81,65.5833,-171.0000,2.46,12.77
...,...,...,...,...,...,...,...,...,...,...
495,495,mangrol,0,IN,1616295556,63,21.1167,70.1167,77.54,10.76
496,496,shubarshi,100,KZ,1616295557,95,48.5872,57.1869,21.06,7.09
497,497,taft,0,IR,1616295557,21,31.7471,54.2088,48.20,1.05
498,498,kenai,40,US,1616295557,58,60.5544,-151.2583,24.80,24.16


In [5]:
# Use gmaps

gmaps.configure(api_key=maps_key)

# Locations
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)

In [6]:
# Plotting the heatmap

fig = gmaps.figure()

# Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# zero cloudiness
no_cloud = weather_df[weather_df["Cloudiness"] == 0]

# max temp above 70 degrees
above_70_degrees = no_cloud[no_cloud["Max Temp"] > 70]

# max temp below 80 degrees F
below_80_degrees = above_70_degrees[above_70_degrees["Max Temp"] < 80]

# wind speed below 10 mph
wind_below_10 = below_80_degrees[below_80_degrees["Wind Speed"] < 10]

# humidity below 50 %
below_50 = wind_below_10[wind_below_10["Humidity"] < 50]


# Index
indexed_below_50 = below_50.reset_index()
del indexed_below_50["index"]
indexed_below_50

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,198,esperance,0,AU,1616295414,44,-33.8667,121.900,78.80,9.22
1,450,aswan,0,EG,1616295538,22,24.0934,32.907,73.27,4.09


In [12]:
# Heat maps

vacation_locations = indexed_below_60[["Lat", "Lng"]]

vacation_humidity = indexed_below_60["Humidity"].astype(float)

# Plot Heatmap
vacation_fig = gmaps.figure()

# Create heat layer
vacation_heat_layer = gmaps.heatmap_layer(vacation_locations, weights=vacation_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vacation_fig.add_layer(vacation_heat_layer)

# Display figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Looping through to find nearest hotels

# Hotel variable
hotels = []

# get nearest hotel
for city in range(len(indexed_below_50["City"])):

    lat = indexed_below_50.loc[city]["Lat"]
    lng = indexed_below_50.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": maps_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_below_50["Nearest Hotel"] = hotels
indexed_below_50

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Nearest Hotel
0,198,esperance,0,AU,1616295414,44,-33.8667,121.900,78.80,9.22,Hospitality Esperance
1,450,aswan,0,EG,1616295538,22,24.0934,32.907,73.27,4.09,Nuba Nile


In [15]:
# Add hotel marks to heatmap

heat_box = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [heat_box.format(**row) for index, row in indexed_below_50.iterrows()]
locations = indexed_below_50[["Lat", "Lng"]]

In [16]:
# Add markers
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))